In [33]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import fitz

In [34]:
# model = SentenceTransformer('sentence-transformers/average_word_embeddings_glove.6B.300d')
model = SentenceTransformer('thenlper/gte-base')

In [35]:
LABELS_FOR_BLOCKS = ["Education", "Skills", "Experience", "Personal", "Good day"]
BLOCK_LABELS_EMBEDDINGS = model.encode(LABELS_FOR_BLOCKS)
BLOCK_LABELS_EMBEDDINGS = [(i, j) for i, j in zip(LABELS_FOR_BLOCKS, BLOCK_LABELS_EMBEDDINGS)]

In [36]:
def process_pdf_file(filename):
    """Получает весь текст из pdf"""
    doc = fitz.open(filename)
    blocks = []
    for page in doc:
        for block in page.get_text_blocks():
            if block[4].startswith("<image:") or "Aspose" in block[4]:
                continue
            blocks.append(block[4])
    return blocks

In [37]:
df = pd.read_csv("../csv/text.csv").drop(columns=["Unnamed: 0"])
df = df[~df["Filename"].str.contains(".doc")]

In [27]:
def get_most_similar_labels_for_block(block):
    block_emb = model.encode(block)
    label = sorted(BLOCK_LABELS_EMBEDDINGS, key=lambda x: cos_sim(block_emb, x[1]),
                   reverse=True)
    return label[0][0]

In [31]:
blocks = process_pdf_file("../" + df["Filename"].iloc[10])

In [32]:
with open("blocks_labels.txt", "w", encoding="utf-8") as file:
    for i in blocks:
        label = get_most_similar_labels_for_block(i)
        file.write(i + "\n")
        file.write(f"LABEL: {label}")
        file.write("-" * 50)
        file.write("\n\n\n\n")